# Line Plots

## 2D line plots

This example plots a sine wave, cosine wave, and ricker wavelet and demonstrates how Graphic Features can be modified by slicing!

In [ ]:
import fastplotlib as fpl
from fastplotlib.graphics.selectors import Synchronizer

import numpy as np
from sidecar import Sidecar
from ipywidgets import VBox
from itertools import product

### First generate some data.

In [ ]:
# linspace, create 100 evenly spaced x values from -10 to 10
xs = np.linspace(-10, 10, 100)
# sine wave
ys = np.sin(xs)
sine = np.dstack([xs, ys])[0]

# cosine wave
ys = np.cos(xs) + 5
cosine = np.dstack([xs, ys])[0]

# sinc function
a = 0.5
ys = np.sinc(xs) * 3 + 8
sinc = np.dstack([xs, ys])[0]

### We will plot all of it on the same plot. Each line plot will be an individual Graphic, you can have any combination of graphics on a plot.

In [ ]:
# Create a plot instance
plot_l = fpl.Plot()

# plot sine wave, use a single color
sine_graphic = plot_l.add_line(data=sine, thickness=5, colors="magenta")

# you can also use colormaps for lines!
cosine_graphic = plot_l.add_line(data=cosine, thickness=12, cmap="autumn")

# or a list of colors for each datapoint
colors = ["r"] * 25 + ["purple"] * 25 + ["y"] * 25 + ["b"] * 25
sinc_graphic = plot_l.add_line(data=sinc, thickness=5, colors = colors)

sc = Sidecar(title="lines")

with sc: 
    display(plot_l.show())

### "stretching" the camera, useful for large timeseries data

Set `maintain_aspect = False` on a camera, and then use the right mouse button and move the mouse to stretch and squeeze the view!

You can also click the `1:1` button to toggle this.

In [ ]:
plot_l.camera.maintain_aspect = False

### reset the plot area

In [ ]:
plot_l.auto_scale(maintain_aspect=True)

### Graphic features support slicing! :D

In [ ]:
# indexing of colors
cosine_graphic.colors[:15] = "magenta"

In [ ]:
cosine_graphic.colors[90:] = "red"

In [ ]:
cosine_graphic.colors[60] = "w"

In [ ]:
# indexing to assign colormaps to entire lines or segments
sinc_graphic.cmap[10:50] = "gray"
sine_graphic.cmap = "seismic"

In [ ]:
# more complex indexing, set the blue value directly from an array
cosine_graphic.colors[65:90, 0] = np.linspace(0, 1, 90-65)

### You can capture changes to a graphic feature as events

In [ ]:
def callback_func(event_data):
    print(event_data)

# Will print event data when the color changes
cosine_graphic.colors.add_event_handler(callback_func)

In [ ]:
# more complex indexing of colors
# from point 15 - 30, set every 3rd point as "cyan"
cosine_graphic.colors[15:50:3] = "cyan"

### Graphic data is also indexable

In [ ]:
cosine_graphic.data[10:50:5, :2] = sine[10:50:5]

In [ ]:
cosine_graphic.data[90:, 1] = 7

In [ ]:
cosine_graphic.data[0] = np.array([[-10, 0, 0]])

In [ ]:
plot_l.close()
sc.close()

### colormaps

In [ ]:
plot = fpl.Plot()

plot.add_line(sine, thickness=10, name="sine")

sc = Sidecar(title="line plot")

with sc:
    display(plot.show())

In [ ]:
plot["sine"].cmap = "jet"

In [ ]:
plot["sine"].cmap.values = sine[:, 1]

In [ ]:
plot["sine"].cmap.values = cosine[:, 1]

In [ ]:
plot["sine"].cmap = "viridis"

Qualitative data

In [ ]:
cmap_values = [0] * 25 + [1] * 5 + [2] * 50 + [3] * 20
plot["sine"].cmap.values = cmap_values

Qualitative cmap

In [ ]:
plot["sine"].cmap = "tab10"

In [ ]:
plot.close()
sc.close()

## Line Collections

Generate some data

In [ ]:
def make_circle(center, radius: float, n_points: int = 75) -> np.ndarray:
    theta = np.linspace(0, 2 * np.pi, n_points)
    xs = radius * np.sin(theta)
    ys = radius * np.cos(theta)

    return np.column_stack([xs, ys]) + center


spatial_dims = (50, 50)

# this makes 16 circles, so we can create 16 cmap values, so it will use these values to set the
# color of the line based by using the cmap as a LUT with the corresponding cmap_value
circles = list()
for center in product(range(0, spatial_dims[0], 15), range(0, spatial_dims[1], 15)):
    circles.append(make_circle(center, 5, n_points=75))

pos_xy = np.vstack(circles)

### Qualitative cmap

Useful for clustering or classification

In [ ]:
plot = fpl.Plot()

# highest values, lowest values, mid-high values, mid values
cmap_values = [
    0, 1, 1, 2,
    0, 0, 1, 1,
    2, 2, 3, 3,
    1, 1, 1, 5
]

plot.add_line_collection(circles, cmap="tab10", cmap_values=cmap_values, thickness=5)

sc = Sidecar(title="lines collection")

with sc:
    display(plot.show())

In [ ]:
plot.close()
sc.close()

### Set colors directly

In [ ]:
plot = fpl.Plot()

colors = ["green"] * 4 + ["red"] * 4 + ["yellow"] * 4 + ["w"] * 4

plot.add_line_collection(circles, colors=colors, thickness=5)

sc = Sidecar(title="lines collection")

with sc:
    display(plot.show())

In [ ]:
plot.close()
sc.close()

## neuro data example

In [ ]:
# load in movie 
movie = np.load('/home/clewis7/repos/fastplotlib-scipy2023/data/rcm.npy')

# load in identified nuerons
contours = np.load('/home/clewis7/repos/fastplotlib-scipy2023/data/contours.npy', allow_pickle=True)

In [ ]:
# for the image data and contours
cnmf_iw = fpl.ImageWidget(movie, vmin_vmax_sliders=True, cmap="gray")

# stack the plots and show them in sidecar
sc = Sidecar(title="good/bad comps")

with sc:
    display(cnmf_iw.show())

In [ ]:
cnmf_iw.vmin_vmax_sliders[0].value = (-5, 15)

### Colors based on classifier output

In [ ]:
# indices of accepted_ixs components
accepted_ixs = np.load('/home/clewis7/repos/fastplotlib-scipy2023/data/good_ixs.npy')

In [ ]:
classifier = np.zeros(len(contours), dtype=int)
classifier[accepted_ixs] = 1

classifier

In [ ]:
# add contours to the plot within the widget
contours_graphic = cnmf_iw.gridplot[0,0].add_line_collection(
    contours, 
    cmap="Set1",
    cmap_values=classifier,
    thickness=3, 
    name="contours"
)

**Red: class 0 - rejected**

**Blue: class 1 - accepted**

### cmaps for quantitative metrics 

#### CNN classifier output

In [ ]:
# load in CNN predictions
cnn_preds = np.load('/home/clewis7/repos/fastplotlib-scipy2023/data/cnn_preds.npy')

In [ ]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

cnn_preds

In [ ]:
contours_graphic.cmap = "spring"
contours_graphic.cmap_values = cnn_preds

**low confidence: purple**

**high confidence: yellow**

#### another quantitative measure

In [ ]:
# load in r_vals
r_vals = np.load('/home/clewis7/repos/fastplotlib-scipy2023/data/r_vals.npy')

contours_graphic.cmap_values = r_vals

#### yet another measure, signal-to-noise measure

In [ ]:
# load in SNR comps
snr_comps = np.load('/home/clewis7/repos/fastplotlib-scipy2023/data/SNR_comps.npy')

contours_graphic.cmap_values = np.log10(snr_comps)

**low signal to noise: purple**

**high signal to noise: yellow**

In [ ]:
cnmf_iw.gridplot.close()
sc.close()

## Line Stack - useful for time series data

Generate some sine data

In [ ]:
xs = np.linspace(0, 100, 1000)
# sine wave
ys = np.sin(xs) * 20

# make 25 lines
data = np.vstack([ys] * 25)

In [ ]:
plot = fpl.Plot()

# line stack takes all the same arguments as line collection and behaves similarly
plot.add_line_stack(data, cmap="jet")

sc = Sidecar(title="line stack")

with sc:
    display(plot.show(maintain_aspect=False))

In [ ]:
plot.close()
sc.close()

## Linear Region Selector

Creates a linear region bounded graphic which can be moved along either the x-axis or y-axis.
Allows sub-selecting data from a `Graphic` or from multiple `Graphic` objects.

In [ ]:
def interpolate(subdata: np.ndarray, axis: int):
    """1D interpolation to display within the preallocated data array"""
    x = np.arange(0, zoomed_prealloc)
    xp = np.linspace(0, zoomed_prealloc, subdata.shape[0])
    
    # interpolate to preallocated size
    return np.interp(x, xp, fp=subdata[:, axis])  # use the y-values

In [ ]:
# data to plot
xs = np.linspace(0, 100, 1_000)
sine = np.sin(xs) * 20
cosine = np.cos(xs) * 20

plot = fpl.GridPlot((5, 1))

# preallocated size for zoomed data
zoomed_prealloc = 1_000
zoomed_init = np.column_stack([np.arange(zoomed_prealloc), np.random.rand(zoomed_prealloc)])

# sines and cosines
sines = [sine] * 2
cosines = [cosine] * 2

# make line stack
line_stack = plot[0, 0].add_line_stack(sines + cosines, separation=50)

# make selector
selector = line_stack.add_linear_region_selector()

# populate subplots with preallocated graphics
for i, subplot in enumerate(plot):
    if i == 0:
        # skip the first one
        continue
    # make line graphics for displaying zoomed data
    subplot.add_line(zoomed_init, name="zoomed")


def update_zoomed_subplots(ev):
    """update the zoomed subplots"""
    zoomed_data = selector.get_selected_data()
    
    for i in range(len(zoomed_data)):
        data = interpolate(zoomed_data[i], axis=1)
        plot[i + 1, 0]["zoomed"].data = data
        plot[i + 1, 0].auto_scale()


selector.selection.add_event_handler(update_zoomed_subplots)
plot.show()
sc = Sidecar(title="linear region selector")

with sc: 
    display(plot.show())

In [ ]:
plot.close()
sc.close()

## Heatmap

In [ ]:
temporal = np.load('/home/clewis7/repos/fastplotlib-scipy2023/data/temporal.npy')

In [ ]:
# create plot
plot = fpl.GridPlot(shape=(1,2), names=[["heatmap", "timeseries"]])

def normalize(array):
    out = np.zeros(array.shape, dtype=array.dtype)
    for i in range(array.shape[0]):
        a = array[i]
        out[i] = ((a - np.min(a)) / (np.max(a - np.min(a))))
        
    return out

temporal_norm = normalize(temporal)
        
# add temporal traces as heatmap
heatmap = plot[0,0].add_heatmap(temporal_norm)

# add temporal traces as line stack
temporal_stack = plot[0,1].add_line_stack(temporal, colors="magenta")

for subplot in plot:
    subplot.camera.maintain_aspect = False

# heatmap support the same selectors as lines!
heatmap_ls = heatmap.add_linear_selector()
temporal_stack_ls = temporal_stack.add_linear_selector()
temporal_stack_ls.position_z = 4

# the corresponding imaging data and some callbacks to update frames
cnmf_iw = fpl.ImageWidget(movie, vmin_vmax_sliders=True, cmap="gray")

# update the linear selectors with current frame index
def update_linear_selector(change):
    ix = change["new"]
    
    heatmap_ls.selection = ix
    temporal_stack_ls.selection = ix
    
def update_ipywidget(ev):
    ix = ev.pick_info["selected_index"]
    # for line collection sends as list
    if not isinstance(ix, int):
        ix = ix[0]
    cnmf_iw.sliders["t"].value = ix
    
cnmf_iw.sliders["t"].observe(update_linear_selector, "value")
heatmap_ls.selection.add_event_handler(update_ipywidget)
temporal_stack_ls.selection.add_event_handler(update_ipywidget)

    
sc = Sidecar(title="heatmap interactive")

with sc:
    display(VBox([cnmf_iw.show(), plot.show()]))

In [ ]:
cnmf_iw.vmin_vmax_sliders[0].value = (-5, 15)

In [ ]:
plot.close()
sc.close()